In [4]:
# 模型保存与加载
# 一、概述
'''
飞桨框架推荐使用的模型保存与加载基础 API 主要包括：

# 这两个用于训练调优场景（动态图）
paddle.save

paddle.load 

# 这两个用于推理部署场景 （动态图）
paddle.jit.save

paddle.jit.load

模型保存与加载高层 API 主要包括：

paddle.Model.save

paddle.Model.load
'''

# 二、用于训练调优场景
## 2.1 保存和加载机制介绍


## 2.2 使用基础 API
import numpy as np
import paddle
import paddle.nn as nn
import paddle.optimizer as opt

BATCH_SIZE = 16
BATCH_NUM = 4
EPOCH_NUM = 4

IMAGE_SIZE = 784
CLASS_NUM = 10

final_checkpoint = dict()

# 定义一个随机数据集
class RandomDataset(paddle.io.Dataset):
    def __init__(self, num_samples):
        self.num_samples = num_samples

    def __getitem__(self, idx):
        image = np.random.random([IMAGE_SIZE]).astype('float32')
        label = np.random.randint(0, CLASS_NUM - 1, (1, )).astype('int64')
        return image, label

    def __len__(self):
        return self.num_samples

class LinearNet(nn.Layer):
    def __init__(self):
        super(LinearNet, self).__init__()
        self._linear = nn.Linear(IMAGE_SIZE, CLASS_NUM)

    def forward(self, x):
        return self._linear(x)

def train(layer, loader, loss_fn, opt):
    for epoch_id in range(EPOCH_NUM):
        for batch_id, (image, label) in enumerate(loader()):
            out = layer(image)
            loss = loss_fn(out, label)
            loss.backward()
            opt.step()
            opt.clear_grad()
            print("Epoch {} batch {}: loss = {}".format(
                epoch_id, batch_id, np.mean(loss.numpy())))
        # 最后一个epoch保存检查点checkpoint
        if epoch_id == EPOCH_NUM - 1:
            final_checkpoint["epoch"] = epoch_id
            final_checkpoint["loss"] = loss
            


# 创建网络、loss和优化器
layer = LinearNet()
loss_fn = nn.CrossEntropyLoss()
adam = opt.Adam(learning_rate=0.001, parameters=layer.parameters())

# 创建用于载入数据的DataLoader
dataset = RandomDataset(BATCH_NUM * BATCH_SIZE)
loader = paddle.io.DataLoader(dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=2)

# 开始训练
train(layer, loader, loss_fn, adam)

### 2.2.1 保存动态图模型
# 保存Layer参数
paddle.save(layer.state_dict(), "linear_net.pdparams")
# 保存优化器参数
paddle.save(adam.state_dict(), "adam.pdopt")
# 保存检查点checkpoint信息
paddle.save(final_checkpoint, "final_checkpoint.pkl")

### 2.2.2 加载动态图模型
# 载入模型参数、优化器参数和最后一个epoch保存的检查点
layer_state_dict = paddle.load("linear_net.pdparams")
opt_state_dict = paddle.load("adam.pdopt")
final_checkpoint_dict = paddle.load("final_checkpoint.pkl")

# 将load后的参数与模型关联起来
layer.set_state_dict(layer_state_dict)
adam.set_state_dict(opt_state_dict)

# 打印出来之前保存的 checkpoint 信息
print("Loaded Final Checkpoint. Epoch : {}, Loss : {}".format(final_checkpoint_dict["epoch"], final_checkpoint_dict["loss"].numpy()))

## 2.3 使用高层 API
# 略

Epoch 0 batch 0: loss = 2.4163055419921875
Epoch 0 batch 1: loss = 2.3251852989196777
Epoch 0 batch 2: loss = 2.8837764263153076
Epoch 0 batch 3: loss = 2.4148805141448975
Epoch 1 batch 0: loss = 2.263458251953125
Epoch 1 batch 1: loss = 2.3448681831359863
Epoch 1 batch 2: loss = 2.1255745887756348
Epoch 1 batch 3: loss = 1.9861130714416504
Epoch 2 batch 0: loss = 2.213273048400879
Epoch 2 batch 1: loss = 2.3677144050598145
Epoch 2 batch 2: loss = 2.6588258743286133
Epoch 2 batch 3: loss = 2.394914150238037
Epoch 3 batch 0: loss = 2.5451154708862305
Epoch 3 batch 1: loss = 2.485877275466919
Epoch 3 batch 2: loss = 2.5099472999572754
Epoch 3 batch 3: loss = 2.627532482147217
Loaded Final Checkpoint. Epoch : 3, Loss : [2.6275325]


In [ ]:
# 四、其他场景
## 4.1 旧版本格式兼容载入

## 4.2 静态图模型的保存与加载
'''
若仅需要保存/载入模型的参数用于训练调优场景，可以使用 paddle.save/paddle.load 结合静态图模型 Program 的 state_dict 达成目的。
也支持保存整个模型，可以使用 paddle.save 将 Program 和state_dict 都保存下来。高层 API 兼容了动态图和静态图，因此Paddle.Model.save和Paddle.Model.load也兼容了动、静态图的保存和加载。

若需保存推理模型用于模型部署场景，则可以通过 paddle.static.save_inference_model、paddle.static.load_inference_model实现。
'''

# 略
